In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import eyecite
from datetime import datetime
from markdown import markdown
from IPython.display import Markdown, display, HTML
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
load_dotenv()
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from typing import Tuple, Optional
from src.schema.decisions import Decision
from src.schema.decisions import extract_parallel_citation_context, filter_decisions_by_forward_citations
from src.doc_store.cap_api import CAPClient
from src.doc_store.cl_client  import CourtListenerClient
from src.utils.citations import create_highlighted_citation
from src.doc_store.base import process_citations_with_progress
from src.doc_store.utils import (
    load_models_from_json, 
    save_models_to_json, 
    looks_like_case_law_link, 
    filter_decisions_with_text,
)

In [3]:
top_citations = pd.read_excel("./data/tables/top_citations.xlsx")
top_citations["cap_ind"] = top_citations["link"].apply(looks_like_case_law_link)
Markdown(top_citations.to_markdown())

|    | Citation              |   Count | link                                                      | cap_ind   |
|---:|:----------------------|--------:|:----------------------------------------------------------|:----------|
|  0 | 120 Cal. Rptr. 2d 162 |     226 | https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162 | False     |
|  1 | 908 A.2d 888          |     197 | https://cite.case.law/a2d/908/888                         | True      |
|  2 | 750 P.2d 297          |     195 | https://cite.case.law/p2d/750/297                         | True      |
|  3 | 121 F.3d 496          |     190 | https://cite.case.law/f3d/121/496                         | True      |
|  4 | 268 S.W.3d 487        |     187 | https://cite.case.law/sw3d/268/487                        | True      |
|  5 | 693 P.2d 796          |     186 | https://cite.case.law/p2d/693/796                         | True      |
|  6 | 36 Cal. App. 4th 500  |     184 | https://cite.case.law/cal-app-4th/36/500                  | True      |
|  7 | 40 P.2d 311           |     182 | https://cite.case.law/p2d/40/311                          | True      |
|  8 | 159 F. Supp. 2d 1196  |     178 | https://cite.case.law/f-supp-2d/159/1196                  | True      |
|  9 | 44 Cal. Rptr. 2d 370  |     177 | https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370  | False     |

In [4]:
cl_client = CourtListenerClient()
cap_client = CAPClient()

In [5]:

def fetch_case(citation: str) -> Tuple[Optional[Decision], bool]:
    """
    Fetches a case by citation, trying CAPClient first and falling back to CourtListenerClient if necessary.

    Args:
        citation (str): The citation string to search for.

    Returns:
        Tuple[Optional[Decision], bool]: A tuple containing the Decision model (or None if not found) and a boolean indicating if the case was found in CAP (True) or CourtListener (False).
    """
    try:
        case = cap_client.read_cite(citation)
        return case, True
    except (ValueError, IndexError):
        try:
            cite = cl_client.find_cite(citation)
            case = cl_client.fetch_case_by_id(cite.id)
            return case, False
        except:
            return None, False

def update_case_counts(case: Decision, is_cap: bool) -> Decision:
    """
    Updates the Decision model with counts of "cites to" and "cited by" cases.

    Args:
        case (Decision): The Decision model to update.
        is_cap (bool): Indicates if the case was found in CAP (True) or CourtListener (False).

    Returns:
        Decision: The updated Decision model.
    """
    if is_cap:
        forward_cites = cap_client.get_forward_citation_ids(case.id)
    else:
        forward_cites = cl_client.get_forward_citation_ids(case.id)
    
    case_updated = case.model_copy(update={"forward_citation_ids": forward_cites})
    cites_to_count = len(case_updated.cites_to)
    cited_by_count = len(case_updated.forward_citation_ids)
    
    return case_updated, cites_to_count, cited_by_count


def process_citations(top_citations: pd.DataFrame) -> Tuple[pd.DataFrame, list]:
    """
    Processes citations in a DataFrame, updating each with "cites to" and "cited by" counts, and collects updated Decision objects.

    Args:
        top_citations (pd.DataFrame): The DataFrame containing citations to process.

    Returns:
        Tuple[pd.DataFrame, list]: The updated DataFrame with new counts and a list of updated Decision objects.
    """
    updated_decisions = []
    
    for idx, row in top_citations.iterrows():
        citation = row["Citation"]
        case, is_cap = fetch_case(citation)
        
        if case:
            case_updated, cites_to_count, cited_by_count = update_case_counts(case, is_cap)
            print(f"'{str(case_updated)}' cites to {cites_to_count} cases & been cited {cited_by_count} times.")
            
            updated_decisions.append(case_updated)
        else:
            print(f"Error reading: {citation}")
            cites_to_count = cited_by_count = 0
        
        top_citations.at[idx, 'cites_to_count'] = cites_to_count
        top_citations.at[idx, 'cited_by_count'] = cited_by_count
    
    return top_citations, updated_decisions


In [6]:
top_citations_updated, decisions_list = process_citations(top_citations)
print(len(decisions_list))

'Cunningham, 120 Cal. Rptr. 2d 162 (2002-04-30)' cites to 22 cases & been cited 7 times.
'Kvaerner Metals Division of Kvaerner U.S., Inc. v. Commercial Union Insurance, 589 Pa. 317 (2006-10-25)' cites to 37 cases & been cited 100 times.
'Isaacson, 750 P.2d 297 (1988-03-07)' cites to 0 cases & been cited 0 times.
'National Steel Corp. v. Golden Eagle Insurance, 121 F.3d 496 (1997-08-06)' cites to 49 cases & been cited 41 times.
'Zurich American Insurance Co. v. Nokia, Inc., 268 S.W.3d 487 (2008-08-29)' cites to 54 cases & been cited 97 times.
'Brandt, 693 P.2d 796 (1985-01-28)' cites to 1 cases & been cited 20 times.
'Pruyn v. Agricultural Insurance, 36 Cal. App. 4th 500 (1995-06-30)' cites to 93 cases & been cited 62 times.
Error reading: 40 P.2d 311
'Everett Associates, Inc. v. Transcontinental Insurance, 159 F. Supp. 2d 1196 (2001-08-28)' cites to 117 cases & been cited 7 times.
'Waller, 900 P.2d 619 (1995-08-28)' cites to 14 cases & been cited 20 times.
9


In [7]:
Markdown(top_citations_updated.to_markdown())

|    | Citation              |   Count | link                                                      | cap_ind   |   cites_to_count |   cited_by_count |
|---:|:----------------------|--------:|:----------------------------------------------------------|:----------|-----------------:|-----------------:|
|  0 | 120 Cal. Rptr. 2d 162 |     226 | https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162 | False     |               22 |                7 |
|  1 | 908 A.2d 888          |     197 | https://cite.case.law/a2d/908/888                         | True      |               37 |              100 |
|  2 | 750 P.2d 297          |     195 | https://cite.case.law/p2d/750/297                         | True      |                0 |                0 |
|  3 | 121 F.3d 496          |     190 | https://cite.case.law/f3d/121/496                         | True      |               49 |               41 |
|  4 | 268 S.W.3d 487        |     187 | https://cite.case.law/sw3d/268/487                        | True      |               54 |               97 |
|  5 | 693 P.2d 796          |     186 | https://cite.case.law/p2d/693/796                         | True      |                1 |               20 |
|  6 | 36 Cal. App. 4th 500  |     184 | https://cite.case.law/cal-app-4th/36/500                  | True      |               93 |               62 |
|  7 | 40 P.2d 311           |     182 | https://cite.case.law/p2d/40/311                          | True      |                0 |                0 |
|  8 | 159 F. Supp. 2d 1196  |     178 | https://cite.case.law/f-supp-2d/159/1196                  | True      |              117 |                7 |
|  9 | 44 Cal. Rptr. 2d 370  |     177 | https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370  | False     |               14 |               20 |

In [8]:
cl_cites = top_citations_updated[top_citations_updated["cap_ind"]==False]["Citation"].tolist()
cap_cites = [
    "121 F.3d 496",
    "268 S.W.3d 487",
]
len(cl_cites), len(cap_cites)

(2, 2)

In [9]:
cl_cases = []

for citation in cl_cites:
    cite = cl_client.find_cite(citation)
    forward_decisions = cl_client.read_forward_citations(cite.id)
    cl_cases.extend(forward_decisions)
    
print(len(cl_cases))    

Fetching forward citation ID: 2441985
Citation: Btr Hampstead, LLC v. Source Interlink, 5 A.3d 142, 194 Md. App. 538 (Md. Ct. Spec. App. 2010)
Fetching forward citation ID: 2439427
Citation: Brucia v. Hartford Acc. and Indem., 307 F. Supp. 2d 1079 (N.D. Cal. 2003)
Fetching forward citation ID: 2251305
Citation: Davis v. Farmers Ins. Group, 134 Cal. App. 4th 100, 35 Cal. Rptr. 3d 738 (Cal. Ct. App. 2005)
Fetching forward citation ID: 2740011
Citation: Baek v. Continental Casualty Co.,  (Cal. Ct. App. 2014)
Fetching forward citation ID: 2473426
Citation: Lakeland Village Owners Association v. Great American Insurance Company, 727 F. Supp. 2d 887 (E.D. Cal. 2010)
Fetching forward citation ID: 2276439
Citation: Mercury Ins. Co. v. Ayala, 11 Cal. Rptr. 3d 158, 116 Cal. App. 4th 1198 (Cal. Ct. App. 2004)
Fetching forward citation ID: 2273431
Citation: BAROCO WEST, INC. v. Scottsdale Ins. Co., 110 Cal. App. 4th 96, 1 Cal. Rptr. 3d 464 (Cal. Ct. App. 2003)
Fetching forward citation ID: 1462146

In [15]:
cite = cap_client.read_cite(cap_cites[0])

In [16]:
read_id = cap_client.read_id(cite.id)

In [20]:
read_id.text_clean

'NATIONAL STEEL CORPORATION, a Delaware corporation, Plaintiff-Appellee, v. GOLDEN EAGLE INSURANCE COMPANY, Defendant-Appellant. No. 95-17291. United States Court of Appeals, Ninth Circuit. Argued and Submitted July 17, 1997. Decided Aug. 6, 1997. *497 Irving H. Greines and Marc J. Poster, Greines, Martin, Stein, & Richland, Beverly Hills, CA, for defendant-appellant. *498 John L. Condrey, Gordon & Rees, San Francisco, CA, for plaintiff-appellee. Before: FARRIS and TASHIMA, Circuit Judges, and STAGG, Senior District Judge. * * The Honorable Tom Slagg, Senior United States District Judge for the Western District of Louisiana, sitting by designation. FARRIS, Circuit Judge: Golden Eagle appeals a district court summary judgment order holding Golden Eagle liable for the stipulated settlement reached by Golden Eagle’s insured after Golden Eagle denied defense to the insured because the claim did not fall with the insured’s policy. The district court had jurisdiction pursuant to 28 U.S.C. § 

In [24]:
cap_cases = []

for citation in cap_cites:
    try:
        cite = cap_client.read_cite(citation)
        forward_decisions = cap_client.read_forward_citations(cite.id)
        cap_cases.extend(forward_decisions)
    except IndexError as e:
        print(f"Error: {e}")
        pass
    
print(len(cap_cases))

Fetching forward citation ID: 8971904


Citation: Bassidji v. Goe, 413 F.3d 928 (2005-06-15)
Fetching forward citation ID: 5388163
Citation: Atmel Corp. v. St. Paul Fire & Marine, 426 F. Supp. 2d 1039 (2005-10-11)
Fetching forward citation ID: 5713928
Citation: Smith v. Pacific Bell Telephone Co., 649 F. Supp. 2d 1073 (2009-08-12)
Fetching forward citation ID: 9499657
Citation: Yates v. Rexton, Inc., 267 F.3d 793 (2001-10-05)
Fetching forward citation ID: 3787146
Citation: Smith v. Pacific Bell Telephone Co., 662 F. Supp. 2d 1199 (2009-08-12)
Fetching forward citation ID: 4127501
Citation: Carlson v. Century Surety Co., 832 F. Supp. 2d 1086 (2011-05-27)
Fetching forward citation ID: 9169166
Citation: Church Mutual Insurance v. United States Liability Insurance, 347 F. Supp. 2d 880 (2004-04-26)
Fetching forward citation ID: 8448529
Citation: Boyd v. City of Oakland, 458 F. Supp. 2d 1015 (2006-10-17)
Fetching forward citation ID: 5632914
Citation: Waimea Bay Associates One, LLC v. Young, 438 F. Supp. 2d 1186 (2006-06-05)
Fetch

In [47]:
from itertools import chain

case_list = list(chain(decisions_list, cl_cases, cap_cases))
len(case_list)

174

In [66]:
decisions_list[1].model_dump()

{'decision_date': '2006-10-25',
 'name': 'KVAERNER METALS DIVISION OF KVAERNER U.S., INC. f/k/a Kvaerner Davy Division of Kvaerner U.S. Inc. f/k/a Davy International, A Division of Trafalgar House, Inc. and Kvaerner Songer, Inc., f/k/a Kvaerner Davy Songer, Inc. f/k/a Davy Songer, Inc., Together A Joint Venture; Kvaerner Metals Division of Kvaerner U.S., Inc. f/k/a Kvaerner Davy Division of Kvaerner U.S. Inc f/k/a Davy International, A Division of Trafalgar House, Inc., Kvaerner Songer, Inc., f/k/a Kvaerner Davy Songer, Inc. f/k/a Davy Songer, Inc.; Kvaerner Public Liability Company f/k/a Trafalgar House Public Liability Company; and Kvaerner ASA v. COMMERCIAL UNION INSURANCE COMPANY, Lexington Insurance Company, National Union Fire Insurance Company of Pittsburgh, PA, Bethlehem Steel Corporation, Thyssen Still Otto Anlagentechnick GMBH, Successor-in-interest to Still Otto, GMBH. Appeal of National Union Fire Insurance Company Of Pittsburgh, PA.',
 'name_abbreviation': 'Kvaerner Metals

In [26]:
top_four = cl_cites + cap_cites
top_four_df = pd.DataFrame({"Citation": top_four})
top_four_updated, decisions = process_citations(top_four_df)
Markdown(top_four_updated.to_markdown())

'Cunningham, 120 Cal. Rptr. 2d 162 (2002-04-30)' cites to 22 cases & been cited 7 times.
'Waller, 900 P.2d 619 (1995-08-28)' cites to 14 cases & been cited 20 times.
'National Steel Corp. v. Golden Eagle Insurance, 121 F.3d 496 (1997-08-06)' cites to 49 cases & been cited 41 times.
'Zurich American Insurance Co. v. Nokia, Inc., 268 S.W.3d 487 (2008-08-29)' cites to 54 cases & been cited 97 times.


|    | Citation              |   cites_to_count |   cited_by_count |
|---:|:----------------------|-----------------:|-----------------:|
|  0 | 120 Cal. Rptr. 2d 162 |               22 |                7 |
|  1 | 44 Cal. Rptr. 2d 370  |               14 |               20 |
|  2 | 121 F.3d 496          |               49 |               41 |
|  3 | 268 S.W.3d 487        |               54 |               97 |

In [67]:
case_list = decisions + cl_cases + cap_cases
len(case_list)

169

In [68]:
save_models_to_json(case_list, "./data/models/top_four.json")
cases = load_models_from_json(Decision, "./data/models/top_four.json")

In [69]:
forward_cases = filter_decisions_by_forward_citations(cases[0], cases)
len(forward_cases)

7

In [70]:
citations = [citation.cite for citation in cases[0].citations]
citations

['120 Cal. Rptr. 2d 162', '98 Cal. App. 4th 1141']

In [71]:
decisions_context = extract_parallel_citation_context(forward_cases, citations)
len(decisions_context)

7

In [72]:
i = 3

Markdown(create_highlighted_citation(decisions_context[i].context, citations[1]))

To paraphrase Lisa M., a deliberate sexual assault “is fairly attributed not to any peculiar aspect of [a massage enterprise], but only to ‘propinquity and lust.’” (Lisa M., 14 supra, 12 Cal.4th at p. 302.) It therefore cannot be characterized as having occurred “with respect to” the conduct of HMWC’s business. ”2 Baek contends that the phrase “but only with respect to the conduct of [HMWC’s] business” should be read broadly, to connote only a minimal causal connection or incidental relationship. Neither of the cases Baek cites in support of this argument concern sexual misconduct, however, and thus neither stands for the proposition that sexual harassment or sexual assault has the requisite causal connection to Baek’s employment to give rise to coverage under an employer’s CGL policy. (Hartford Casualty Ins. Co. v. Travelers Indemnity Co. (2003) 110 Cal. App.4th 710 , 714 [held: commercial tenant’s CGL insurer owed landlord a duty of defense in premises liability action brought by tenant’s employee’s parents; rejecting insurer’s contention that “‘but only with respect to’” refers only to liability directly caused by landlord]; Acceptance Ins. Co. v. Syufy Enterprises (1999) 69 Cal. App.4th 321 , 328 [building owner was additional insured under contractor’s CGL policy when contractor’s employee was injured while performing repairs on building’s roof; “[employee’s] injury clearly ‘arose out of’ the work he was performing on the roof of Syufy’s building. The relationship between the defective [roof] hatch and the job was more than incidental, in that [employee] could not have done the job without passing through the hatch.” ].) D. Baek’s Additional Contentions Are Without Merit 1. Negligence Baek contends that even if there was no duty to defend with regard to the allegations of sexual assault, there was a duty to defend Jaime W.’s claim of negligence. We do not agree. “The scope of the duty does not depend on the labels given to the causes of action in the third party complaint; instead it rests on whether the alleged facts . . . reveal a possibility that the claim may be covered by the policy.” (Atlantic Mutual Ins. Co. v. J. Lamb, Inc. (2002) 100 Cal.App.4th 1017 , 1034, italics omitted; 2 Having so concluded, we do not address Continental’s additional contention that it did not owe Baek a duty of defense because he did not timely tender his claim. 15 Cunningham v. Universal Underwriters (2002) <mark>98 Cal.App.4th 1141</mark> , 1148 [same].) Here, although Jaime W. asserts a cause of action for negligent massage, the sole facts alleged in support describe intentional conduct—i.e., that Baek “touched, fondled, rubbed, grabbed and squeezed Plaintiff’s breasts, buttocks, inner thighs and genitals, all while making and emitting moans, groans, grunts and other sounds and noises of sexual pleasure.” These alleged acts of sexual misconduct “are, by their very nature, intentional” (Coit Drapery Cleaners, Inc. v. Sequoia Ins. Co. (1993) 14 Cal. App.4th 1595 , 1606)—to paraphrase Shanahan v. State Farm General Ins. Co. (2011) 193 Cal. App.4th 780 , 788, one does not “accidentally” sexually fondle another. Thus, the cause of action for negligence did not give rise to a duty to defend. Jaime W.’s complaint also alleges that each defendant negligently hired, trained, and supervised Baek. Although this allegation on its face is alleged against Baek, as well as the other defendants, we conclude that Baek cannot be liable for failing to use due care while supervising himself. (See State Farm Fire & Casualty Co. v. Century Indemnity Co. (1997) 59 Cal. App.4th 648 , 663 [“We next note that State Farm treats Derrick and the other teachers the same with respect to the claims based on the failure to report. However, in doing so, State Farm conveniently ignores obvious and material differences between them. The other teachers’ potential liability arose solely from their failure to properly respond to information about sexual misconduct by another teacher. . . . [¶] [I]f [insurer] had a duty to defend Derrick against claims based on failure to report, what would emerge is a simple way to plead around the caselaw that precludes a duty to defend teachers against claims of molestation. One need only allege that the teacher/molester negligently failed to report the information known to him or her that would lead a reasonable person to suspect a student had been molested.” ].) 2. False Imprisonment Baek contends Continental had a duty to defend because Jaime W.’s complaint contained a cause of action for false imprisonment, for which there is coverage under the Continental policy. Again, we do not agree. Courts have found no duty to defend 16 allegations of nonsexual misconduct that are “inseparably intertwined” with allegations of intentional sexual misconduct.

In [40]:
import nest_asyncio
nest_asyncio.apply()

all_data = await process_citations_with_progress(decisions_context)

100%|██████████| 7/7 [01:17<00:00, 11.10s/it]


In [41]:
data_dicts = [model.model_dump() for model in all_data]
df_res = pd.DataFrame(data_dicts)
Markdown(df_res.to_markdown(index=False))

| citation              | legal_question                                                                                                                                                                                                                       | rule                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | application                                                                                                                                                                                                                                                                                                                                                                                                             | citation_reference                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|:----------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 98 Cal. App. 4th 1141 | Is BTR's action in allowing Fidelitone to take possession of the premises leased to Source Interlink seen as an actual eviction under the legal definition of eviction?                                                              | An eviction is defined as the process or act of legally dispossessing a person of land or rental property. A wrongful eviction occurs when the entity reclaiming the property has no legal right to dispossess the other party of the property.                                                                                                                                                                                                                                                                                        | In the cited case, the act of legally dispossessing someone from their leased land or property is considered an eviction, which supports the notion that if BTR's actions led to Source Interlink being dispossessed of the premises without a rightful claim, it could represent an actual eviction.                                                                                                                   | The citation is used to support the legal definition of an eviction, which is equated to the act or process of legally dispossessing a person of land or rental property. The explanation further describes wrongful eviction as occurring when the party recovering the property has no legal right to dispossess the other party from the property. This definition aligns with the context of assessing whether BTR's actions could be considered an actual eviction under the given circumstances.                              |
| 98 Cal. App. 4th 1141 | Does a nonpossessing tenant's claimed loss related to interference with their right to possession of real property under a lease agreement constitute damage to tangible property?                                                   | A tenant's right to possession does not constitute a tangible property interest until actual possession occurs. A nonpossessing tenant does not have a tangible property interest and cannot allege damage to tangible property.                                                                                                                                                                                                                                                                                                       | The court applied this rule to determine that the plaintiff, a nonpossessing tenant whose loss concerned the interference with its right to possession under the lease, did not suffer damage to tangible property and thus cannot claim for such damage.                                                                                                                                                               | In Cunningham v. Universal Underwriters, the court offered significant insight by explaining that a landlord's failure to deliver possession merely confers upon the tenant the right to abandon the tenancy and sue for damages, but does not establish the tenant as a legal possessor of the property until the transfer of actual possession.                                                                                                                                                                                   |
| 98 Cal. App. 4th 1141 | What are the legal principles regarding the interpretation of insurance policies and the resolution of ambiguities in favor of the insured?                                                                                          | If a policy provision is ambiguous, we resolve the ambiguity in the insureds' favor, consistent with the insureds' reasonable expectations.                                                                                                                                                                                                                                                                                                                                                                                            | The court addresses the interpretation of insurance policy provisions and highlights that in cases of ambiguity, the decision should favor the insured, aligning with their reasonable expectations.                                                                                                                                                                                                                    | The citation is referenced in relation to principles governing the interpretation of insurance policies, especially when addressing ambiguous policy provisions. It emphasizes that ambiguities should be resolved in the insured's favor, reflecting their reasonable expectations.                                                                                                                                                                                                                                                |
| 98 Cal. App. 4th 1141 | Does an allegation of negligence in a lawsuit involving sexual misconduct give rise to a duty to defend under an insurance policy?                                                                                                   | The scope of the duty to defend does not depend on the labels given to the causes of action in the third party complaint; it essentially rests on whether the alleged facts reveal a possibility that the claim may be covered by the policy.                                                                                                                                                                                                                                                                                          | In the context of sexual misconduct, alleged acts described as negligence but involving intentional conduct, such as sexual assault or harassment, do not give rise to a duty to defend because these acts are, by their nature, intentional. A cause of action for negligence based on the described misconduct does not give rise to a duty to defend as it falls outside the coverage of typical insurance policies. | The case of Cunningham v. Universal Underwriters, 98 Cal.App.4th 1141, is cited to support the principle that the determination of a duty to defend in insurance coverage does not depend on the specific labels of the causes of action but on the underlying facts possibly indicating coverage. It specifically addresses the context of intentional conduct described as negligence, underscoring that such conduct, especially when involving sexual misconduct, does not result in a duty to defend under insurance policies. |
| 98 Cal. App. 4th 1141 | In a duty to defend case, does the insured need only to demonstrate the existence of potential for coverage under the policy, or does the insurer have the greater burden of proving that there is no potential for policy coverage? | In a duty to defend case, an insured moving for summary judgment need only show the existence of a potential for coverage, that the underlying claim may fall within policy coverage. On the other hand, an insurer moving for summary judgment must establish the absence of any potential for coverage, meaning that the underlying complaint can by no conceivable theory raise a single issue which could bring it within the policy coverage. The insurer has a greater burden than the insured when moving for summary judgment. | This establishes that in cases regarding the duty to defend, the burden of proof is significantly different depending on whether the party is the insured or the insurer. The insured need only demonstrate a potential for coverage, whereas the insurer must conclusively prove the absence of any possibility of coverage within the defined policy terms.                                                           | The citation is directly referenced to explain the differing burdens of proof between an insured and an insurer in a duty to defend case. It specifies the burden of proof required for each party when moving for summary judgment.                                                                                                                                                                                                                                                                                                |
| 98 Cal. App. 4th 1141 | In a declaratory relief action taken by an insurer after denying claims made under uninsured motorist coverage, does the per person limit of the policy apply to the claims, or does the per accident limit apply?                   | The language within an insurance policy that specifies the liability limit as applicable to each person is unambiguous and applies the per person limit to claims, according to the plain meaning of that language.                                                                                                                                                                                                                                                                                                                    | The court agreed with the insurer that the per person limit, rather than the per accident limit, applies to appellants' claims due to the unambiguous language within the policy.                                                                                                                                                                                                                                       | In Cunningham v. Universal Underwriters, the court found that similar language in liability policies is unambiguous and concluded that the per person limit applies to the claims in question.                                                                                                                                                                                                                                                                                                                                      |
| 98 Cal. App. 4th 1141 | What are the legal considerations involved in determining liability or coverage in cases referenced by 98 Cal. App. 4th 1141?                                                                                                        | A case law or legal precedent that elaborates on the factors or conditions under which liability or coverage is determined in similar cases.                                                                                                                                                                                                                                                                                                                                                                                           | The case reference might detail specific criteria, conditions, or scenarios under which liability or coverage was assessed, providing guidance on how these legal standards are applied in practice.                                                                                                                                                                                                                    | The citation was used to discuss or illustrate a point related to the legal considerations of liability or coverage in a specific context.                                                                                                                                                                                                                                                                                                                                                                                          |

In [74]:
from src.utils.pydantic_utils import *
from lancedb.pydantic import LanceModel, Vector

In [98]:
cap_cases[0].text_clean

'Massoud BASSIDJI, Plaintiff-Appellee, v. Simon Soul Sun GOE, Defendant-Appellant. No. 02-16019. United States Court of Appeals, Ninth Circuit. Argued March 13, 2003. Submitted Feb. 23, 2004. Filed June 15, 2005. *930 Lori A. Lutzker and W. George Wailes, Carr, McClellan, Ingersoll, Thompson & Horn, Burlingame, CA, for the defendant-appellant. Douglas A. Applegate and Mark W. Epstein, Seiler Epstein Ziegler & Applegate LLP, San Francisco, CA, for the plaintiff-appellee. Before: KOZINSKI, GRABER, and BERZON, Circuit Judges. BERZON, Circuit Judge: Executive Order 13,059 (the “Executive Order” or “Order”), 62 Fed.Reg. 44,531 (Aug. 21, 1997), prohibits United States citizens from investing in and trading with Iran. 1 The question we face is whether an American citizen’s guarantees of payments that furthered a trade agreement with an Iranian company are covered by the Executive Order and, if so, whether the guarantees are unenforceable as a result. We conclude that the guarantees were illeg

In [87]:
fields = [
    "id",
    "decision_date",
    "name",
    "judges",
    "attorneys",
    "head_matter",
    "frontend_url", 
    "data",
    "status",
    "forward_citation_ids",
    "cites_to",    
]

In [108]:
cases= filter_decisions_with_text(cases)
len(cases)

167

In [119]:
fields_dicts = []
for i in range(len(cases)):
    try:
        case_dict = extract_fields(cases[i], fields)
        if not isinstance(cases[i].casebody.data, str):
            case_dict["type"] = cases[i].casebody.data.opinions[0].type if cases[i].casebody.data.opinions else None
        else:
            case_dict["type"] = "Missing"
        case_dict["content"] = cases[i].text_clean
        case_dict["html"] = cases[i].text
        case_dict["citation"] = cases[i].citations if cases[i].citations else None
        fields_dicts.append(case_dict)
    except AttributeError as e:
        print(f"{e}")
        pass
df_cases = pd.DataFrame(fields_dicts)

In [124]:
print(f"df cases shape: {df_cases.shape}")
df_cases.sample(2)

df cases shape: (167, 16)


,id,decision_date,name,judges,attorneys,head_matter,frontend_url,status,forward_citation_ids,cites_to,type,content,html,citation,data,tokens
124,9992,2015-06-26,"MCGINNES INDUSTRIAL MAINTENANCE CORPORATION, A...",NaN,None,NaN,https://cite.case.law/sw3d/477/786/,ok,None,"[243 S.W.2d 154, 150 Tex. 513, 46 F.3d 1116, 2...",Missing,"MCGINNES INDUSTRIAL MAINTENANCE CORPORATION, A...","<section class=""casebody"" data-case-id=""320441...",[477 S.W.3d 786],"<section class=""casebody"" data-case-id=""320441...",19983
127,8820,2011-01-07,"VRV DEVELOPMENT L.P., formerly known as VRV De...",NaN,None,NaN,https://cite.case.law/f3d/630/451/,ok,None,"[100 F.2d 441, 955 F.2d 1353, 434 F.3d 165, 56...",Missing,"VRV DEVELOPMENT L.P., formerly known as VRV De...","<section class=""casebody"" data-case-id=""320441...",[630 F.3d 451],"<section class=""casebody"" data-case-id=""320441...",5427


In [121]:
from src.utils.gen_utils import count_tokens

df_cases["tokens"] = df_cases["content"].apply(lambda x: count_tokens(x))

In [122]:
df_cases["tokens"].describe()

count      167.000000
mean      8883.143713
std       6724.815578
min        640.000000
25%       4809.500000
50%       7385.000000
75%      10944.500000
max      54689.000000
Name: tokens, dtype: float64

In [123]:
df_cases.to_excel("./data/tables/cases_with_text.xlsx", index=False)

In [125]:
top_four[0]

'120 Cal. Rptr. 2d 162'

In [126]:
str(cases[0])

'Cunningham, 120 Cal. Rptr. 2d 162 (2002-04-30)'

In [127]:
cited_by_test = cl_client.read_cited_by(cases[0].id)

Fetching case ID: 1160464
Citation: Vons Companies, Inc. v. Seabest Foods, Inc., 14 Cal. 4th 434 (Cal. 1996)
Fetching case ID: 2258291
Citation: General Accident Ins. Co. v. West Am. Ins. Co., 42 Cal. App. 4th 95, 49 Cal. Rptr. 2d 603 (Cal. Ct. App. 1996)
Fetching case ID: 2302806
Citation: Martin Marietta Corp. v. Insurance Co. of N. Am., 40 Cal. App. 4th 1113, 47 Cal. Rptr. 2d 670 (Cal. Ct. App. 1995)
Fetching case ID: 2253301
Citation: Fibreboard Corp. v. Hartford Acc. & Indemnity Co., 16 Cal. App. 4th 492, 20 Cal. Rptr. 2d 376 (Cal. Ct. App. 1993)
Fetching case ID: 2293434
Citation: Hendrickson v. ZURICH AMERICAN INS. CO., 85 Cal. Rptr. 2d 622, 72 Cal. App. 4th 1084 (Cal. Ct. App. 1999)
Fetching case ID: 2144332
Citation: Bartlome v. State Farm Fire & Casualty Co., 208 Cal. App. 3d 1235, 256 Cal. Rptr. 719 (Cal. Ct. App. 1989)
Fetching case ID: 2270816
Citation: Lee v. Placer Title Co., 28 Cal. App. 4th 503, 33 Cal. Rptr. 2d 572 (Cal. Ct. App. 1994)
Fetching case ID: 1158094
Citati

In [128]:
len(cited_by_test)

22

In [129]:
cited_by_test[0].text_clean

'14 Cal.4th 434 (1996) VONS COMPANIES, INC., Cross-complainant and Appellant, \nv. \nSEABEST FOODS, INC., et al., Cross-defendants and Respondents. Docket No. S049039. Supreme Court of California. December 12, 1996. *440 COUNSEL Munger, Tolles & Olson, Gregory P. Stone, Joseph D. Lee, Kristin A. Linsley, David M. Rosenzweig, Susan R. Szabo and Kristin S. Escalante for Cross-complainant and Appellant. Neil, Dymott, Perkins, Brown & Frank, Thomas M. Dymott, Kyle A. Cruse, Crosby, Heafey, Roach & May, Peter W. Davis, James C. Martin, Jacques B. LeBoeuf, Laskero & Associates and Michael G. Roddy for Cross-defendants and Respondents. OPINION GEORGE, C.J. We consider in this case the issue whether California courts may exercise personal jurisdiction over owners of "fast food" restaurant franchises located in another state, in litigation stemming from several incidents of food poisoning that occurred in the foreign jurisdiction. We conclude that the franchisees had sufficient contacts with Ca

In [139]:
decisions[2].id, str(decisions[2])

(285037,
 'National Steel Corp. v. Golden Eagle Insurance, 121 F.3d 496 (1997-08-06)')

In [140]:
len(decisions[2].cites_to)

49

In [141]:
cited_by_test_2 = cap_client.read_cited_by(decisions[2].id)

Fetching case ID: 2282116
Fetched Decision: Clemmer v. Hartford Insurance, 22 Cal. 3d 865 (1978-12-19)
Fetching case ID: 2013849
Fetched Decision: Samson v. Transamerica Insurance, 30 Cal. 3d 220 (1981-11-23)
Fetching case ID: 1310362
Fetched Decision: Aetna Casualty & Surety Co. v. Centennial Insurance, 838 F.2d 346 (1988-02-01)
Fetching case ID: 11318566
Fetched Decision: Salve Regina College v. Russell, 499 U.S. 225 (1991-03-20)
Fetching case ID: 4372662
Fetched Decision: Xebec Development Partners, Ltd. v. National Union Fire Insurance, 12 Cal. App. 4th 501 (1993-01-13)
Fetching case ID: 6207800
Fetched Decision: Celotex Corp. v. Catrett, 477 U.S. 317 (1986-06-25)
Fetching case ID: 2263866
Fetched Decision: Kuchel v. McCormack, 104 Cal. App. 2d 506 (1951-05-31)
Fetching case ID: 4881724
Fetched Decision: Pruyn v. Agricultural Insurance, 36 Cal. App. 4th 500 (1995-06-30)
Fetching case ID: 10513471
Fetched Decision: Koch Industries, Inc. v. Mobil Oil Corp., 968 F.2d 27 (1992-04-07)
F

In [142]:
cited_by_test_2[0].text_clean

'[L.A. No. 30759. Dec. 19, 1978.] MARJORIE J. CLEMMER et al., Plaintiffs and Appellants, v. HARTFORD INSURANCE COMPANY, Defendant and Appellant. *871 Counsel Gibson, Dunn & Crutcher, John H. Sharer and Fred F. Gregory for Plaintiffs and Appellants. Overton, Lyman & Prince, Carl J. Schuck, John D. McCurdy, Valerie Baker and Gwen H. Whitson for Defendants and Appellants. Opinion MANUEL, J. By this action plaintiffs, the widow and the son of a victim of a killing, sue to recover from the liability insurer of the slayer the amount of a wrongful death judgment obtained against him. Following a jury verdict the trial court entered judgment against the defendant insurer, but it subsequently granted defendant’s motion for new trial, tendered on all issues, on the sole issue submitted to the jury—i.e., whether the death in question was caused by a willful act—and denied it on all other issues. Plaintiffs appeal from the aforesaid order granting a limited new trial. Defendant appeals from that p

In [143]:
print(len(cited_by_test))
cited_by_test.extend(cited_by_test_2)
print(len(cited_by_test))

22
47


In [144]:
save_models_to_json(cited_by_test, "./data/models/cited_to.json")